## 3단계 - 오토 라벨링
2단계 `character_crop.ipynb`에서 크롭한 아바타 이미지들의 포즈 라벨링을 자동화하는 코드 구현

In [ ]:
import os
import torch
import torch.nn as nn
from torchvision import transforms, models
from PIL import Image
from tqdm import tqdm

# 1. 경로 설정
base_path = '/home/ssy/maple-vision-search/data/processed'
image_dir = os.path.join(base_path, 'temp_crop')   # 크롭된 이미지들이 있는 폴더
label_dir = os.path.join(base_path, 'temp_label')  # 결과를 저장할 폴더

if not os.path.exists(label_dir):
    os.makedirs(label_dir)

# 2. 모델 로드 및 설정
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 모델 구조 정의
model = models.mobilenet_v3_small(weights=None)
num_features = model.classifier[3].in_features
model.classifier[3] = nn.Linear(num_features, 10) # 학습 시 설정한 10개 클래스

# 가중치(state_dict) 로드
model_weight_path = '/home/ssy/maple-vision-search/weights/best_pose_classifier.pth'
state_dict = torch.load(model_weight_path, map_location=DEVICE)
model.load_state_dict(state_dict)

model = model.to(DEVICE)
model.eval()

# 3. 전처리 설정 
preprocess = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# 4. 추론 및 .txt 파일 생성
image_files = [f for f in os.listdir(image_dir) if f.endswith(('.png', '.jpg'))]

print(f"오토 라벨링 시작: 총 {len(image_files)}개 이미지")

with torch.no_grad():
    for filename in tqdm(image_files):
        img_path = os.path.join(image_dir, filename)
        try:
            # 이미지 로드 및 변환
            image = Image.open(img_path).convert('RGB')
            input_tensor = preprocess(image).unsqueeze(0).to(DEVICE)
            
            # 모델 추론
            outputs = model(input_tensor)
            _, predicted_idx = torch.max(outputs, 1)
            label_index = predicted_idx.item()

            # 파일명.txt 생성 및 결과 기록
            txt_filename = os.path.splitext(filename)[0] + '.txt'
            txt_path = os.path.join(label_dir, txt_filename)

            with open(txt_path, 'w') as f:
                f.write(str(label_index))
                
        except Exception as e:
            print(f"\n[오류] {filename} 처리 중 에러 발생: {e}")

print(f"\n작업 완료: '{label_dir}' 폴더에 10개 클래스 기반 라벨링이 완료되었습니다.")

오토 라벨링 시작: 총 1467개 이미지


100%|██████████| 1467/1467 [00:03<00:00, 401.24it/s]


작업 완료: '/home/ssy/maple-vision-search/data/processed/temp_label' 폴더에 10개 클래스 기반 라벨링이 완료되었습니다.
